In [1]:
# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 引入 hashlib 模組
import hashlib

# 引入 regular expression 工具
import re

In [ ]:
options = webdriver.ChromeOptions()
               
options.add_argument("--start-maximized")         
options.add_argument("--incognito")               
options.add_argument("--disable-popup-blocking ")

driver = webdriver.Chrome(options = options)

listStory = []

folderPath = "/Users/gaoming/Downloads/ChineseBooks"
if not os.path.exists(folderPath):
    os.makedirs(folderPath)
    
def openChrome():
    driver.get("https://www.gutenberg.org/browse/languages/zh")
    sleep(3)
    
def findHref():
    a_hrefs = driver.find_elements(By.CSS_SELECTOR,"ul > li.pgdbetext > a")
    
    for a in a_hrefs:
        link = a.get_attribute('href')
        title = a.get_attribute('innerText')
        regex = r"\d+"
        linkNumber = re.search(regex,link).group()
        filename = "/"+ linkNumber +"-0.txt"
        txtLink = "https://www.gutenberg.org/files/"+linkNumber+filename
        
        listStory.append({
            'title': title,
            'link': txtLink,
            'id': linkNumber
        })
        
def getStory():
    
    for index, obj in enumerate(listStory):
        
        driver.get(obj['link'])
        
        try:
            WebDriverWait(driver, 3).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, 'body > pre')
                )
            )
            story = driver.find_element(By.CSS_SELECTOR,'body > pre').get_attribute('innerText')
        
            story = ''.join(re.findall(r'[\u4E00-\u9FFF，。：「」；、？！『』]+',story))
        
            listStory[index]['story'] = story
            
            fp = open(f"{folderPath}/{listStory[index]['id']}_{listStory[index]['title']}.txt","w",encoding="utf-8")
            fp.write(listStory[index]['story'])
            fp.close
            
            sleep(1)
            
        except TimeoutException:
            continue
            
def quit():
    sleep(3)
    driver.quit()

if __name__ == "__main__":
    openChrome()
    findHref()
    getStory()
    quit()